## Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import folium
import requests
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

## Web Scraping of the `Wikepedia` page

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, "html5lib")
right_table=soup.find('table', class_='wikitable sortable')
#right_table

## Creating DataFrame using the scrapped data

In [3]:
A=[]
B=[]
C=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3:
        A.append((cells[0].find(text=True))[:-1])
        B.append((cells[1].find(text=True))[:-1])
        C.append((cells[2].find(text=True))[:-1])
df=pd.DataFrame(A,columns=['PostalCode'])
df['Borough']=B
df['Neighborhood']=C
print("Old shape of dataframe - ",df.shape)

Old shape of dataframe -  (180, 3)


## Cleaning data for `Borough` and `Neighborhood`

In [4]:
#Removing the 'Not assigned' value of 'Borough'
df.drop(df[df.Borough == 'Not assigned'].index, inplace = True)
#Replacing the 'Not assigned' value of 'Neighborhood' column by 'Borough' column's value
df.loc[df['Neighborhood'] == 'Not assigned', 'Neighborhood'] = df.loc[df['Neighborhood'] == 'Not assigned', 'Borough']
df.reset_index(inplace=True)
df.drop(["index"],axis=1,inplace=True)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## Checking whether is there any `Not assigned` value present in `Neighborhood` column or not

In [5]:
df[df.Neighborhood == 'Not assigned'].head()

,PostalCode,Borough,Neighborhood


## Printing the shape of final dataset

In [6]:
print('New shape of dataframe - ',df.shape)

New shape of dataframe -  (103, 3)


## Reading dataset for lattitude and longitude

In [7]:
postalcode_coor = pd.read_csv('http://cocl.us/Geospatial_data')
new_df = postalcode_coor.rename(columns = {'Postal Code': 'PostalCode'})
new_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## Merging two dataset

In [8]:
merged_data = pd.merge(left=df, right=new_df, left_on='PostalCode', right_on='PostalCode')
merged_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [9]:
merged_data['Borough'].value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
York                 5
East Toronto         5
East York            5
Mississauga          1
Name: Borough, dtype: int64

## Get the latitude and longitude values of Toronto

In [10]:
address = 'Toronto'

geolocator = Nominatim(user_agent="Coursera_Capstone")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


## Create a map of Toronto with neighborhoods superimposed on top

In [11]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(merged_data['Latitude'], merged_data['Longitude'], merged_data['Borough'], merged_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

## Filter only boroughs that contain the word Toronto

In [12]:
# filter borough names that contain the word Toronto
borough_names = list(merged_data.Borough.unique())

borough_with_toronto = []

for x in borough_names:
    if "toronto" in x.lower():
        borough_with_toronto.append(x)
        
borough_with_toronto

['Downtown Toronto', 'East Toronto', 'West Toronto', 'Central Toronto']

In [13]:
# create a new DataFrame with only boroughs that contain the word Toronto
merged_data = merged_data[merged_data['Borough'].isin(borough_with_toronto)].reset_index(drop=True)
print(merged_data.shape)
merged_data.head()

(39, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [14]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(merged_data['Latitude'], merged_data['Longitude'], merged_data['Borough'], merged_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

## Use the Foursquare API to explore the neighborhoods

In [15]:
# define Foursquare Credentials and Version
CLIENT_ID = 'your Foursquare ID' # your Foursquare ID
CLIENT_SECRET = 'your Foursquare Secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: your Foursquare ID
CLIENT_SECRET:your Foursquare Secret


#### Now, let's get the top 100 venues that are within a radius of 500 meters.

In [16]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(merged_data['Latitude'], merged_data['Longitude'], merged_data['PostalCode'], merged_data['Borough'], merged_data['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [17]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(1629, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Cooper Koo Family YMCA,43.653249,-79.358008,Distribution Center
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


#### Let's check how many venues were returned for each PostalCode

In [18]:
venues_df.groupby(["PostalCode", "Borough", "Neighborhood"]).count()

,,,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
PostalCode,Borough,Neighborhood,,,,,,
M4E,East Toronto,The Beaches,5,5,5,5,5,5
M4K,East Toronto,"The Danforth West, Riverdale",42,42,42,42,42,42
M4L,East Toronto,"India Bazaar, The Beaches West",17,17,17,17,17,17
M4M,East Toronto,Studio District,40,40,40,40,40,40
M4N,Central Toronto,Lawrence Park,4,4,4,4,4,4
M4P,Central Toronto,Davisville North,8,8,8,8,8,8
M4R,Central Toronto,"North Toronto West, Lawrence Park",18,18,18,18,18,18
M4S,Central Toronto,Davisville,32,32,32,32,32,32
M4T,Central Toronto,"Moore Park, Summerhill East",2,2,2,2,2,2


#### Let's find out how many unique categories can be curated from all the returned venues

In [19]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 231 uniques categories.


In [20]:
venues_df['VenueCategory'].unique()[:50]

array(['Bakery', 'Coffee Shop', 'Distribution Center', 'Spa',
       'Restaurant', 'Park', 'Pub', 'Breakfast Spot',
       'Gym / Fitness Center', 'Historic Site', 'Farmers Market',
       'Dessert Shop', 'Chocolate Shop', 'Performing Arts Venue',
       'French Restaurant', 'Mexican Restaurant', 'Café', 'Yoga Studio',
       'Theater', 'Event Space', 'Asian Restaurant', 'Shoe Store',
       'Ice Cream Shop', 'Electronics Store', 'Art Gallery', 'Bank',
       'Beer Store', 'Hotel', 'Health Food Store', 'Antique Shop',
       'Italian Restaurant', 'Creperie', 'Arts & Crafts Store',
       'Burrito Place', 'Beer Bar', 'Hobby Shop', 'Diner',
       'Fried Chicken Joint', 'Smoothie Shop', 'Sandwich Place', 'Gym',
       'College Auditorium', 'Bar', 'College Cafeteria',
       'Vegetarian / Vegan Restaurant', 'Music Venue', 'Clothing Store',
       'Tea Room', 'Pizza Place', 'Comic Shop'], dtype=object)

## Analyze Each Area

In [21]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
toronto_onehot['PostalCode'] = venues_df['PostalCode'] 
toronto_onehot['Borough'] = venues_df['Borough'] 
toronto_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1629, 234)


,PostalCode,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [22]:
toronto_grouped = toronto_onehot.groupby(["PostalCode", "Borough", "Neighborhoods"]).mean().reset_index()

print(toronto_grouped.shape)
toronto_grouped

(39, 234)


,PostalCode,Borough,Neighborhoods,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.20000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
1,M4K,East Toronto,"The Danforth West, Riverdale",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.02381,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.023810
2,M4L,East Toronto,"India Bazaar, The Beaches West",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,M4M,East Toronto,Studio District,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.025000,0.00,0.025000
4,M4N,Central Toronto,Lawrence Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
5,M4P,Central Toronto,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.055556
7,M4S,Central Toronto,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.03125,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
8,M4T,Central Toronto,"Moore Park, Summerhill East",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.00000,0.00,0.000000,0.000000,0.062500,0.000000,0.00,0.000000


In [23]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = toronto_grouped['PostalCode']
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    row_categories = toronto_grouped.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

# neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted

(39, 13)


,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,Asian Restaurant,Trail,Pub,Neighborhood,Health Food Store,Dog Run,Distribution Center,Doner Restaurant,Cupcake Shop,Discount Store
1,M4K,East Toronto,"The Danforth West, Riverdale",Greek Restaurant,Italian Restaurant,Coffee Shop,Restaurant,Ice Cream Shop,Furniture / Home Store,Fruit & Vegetable Store,Pub,Pizza Place,Lounge
2,M4L,East Toronto,"India Bazaar, The Beaches West",Sushi Restaurant,Liquor Store,Sandwich Place,Burrito Place,Restaurant,Italian Restaurant,Fast Food Restaurant,Steakhouse,Ice Cream Shop,Fish & Chips Shop
3,M4M,East Toronto,Studio District,Café,Coffee Shop,Gastropub,Bakery,Brewery,American Restaurant,Yoga Studio,Comfort Food Restaurant,Sandwich Place,Cheese Shop
4,M4N,Central Toronto,Lawrence Park,Park,Bus Line,Swim School,Ethiopian Restaurant,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center
5,M4P,Central Toronto,Davisville North,Convenience Store,Breakfast Spot,Hotel,Food & Drink Shop,Sandwich Place,Department Store,Gym / Fitness Center,Park,Doner Restaurant,Dog Run
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",Coffee Shop,Clothing Store,Yoga Studio,Sporting Goods Shop,Fast Food Restaurant,Diner,Metro Station,Mexican Restaurant,Miscellaneous Shop,Chinese Restaurant
7,M4S,Central Toronto,Davisville,Dessert Shop,Sandwich Place,Pizza Place,Coffee Shop,Italian Restaurant,Café,Gym,Sushi Restaurant,Pharmacy,Seafood Restaurant
8,M4T,Central Toronto,"Moore Park, Summerhill East",Park,Summer Camp,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Distribution Center,Discount Store
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",Coffee Shop,Pub,Sushi Restaurant,Pizza Place,Liquor Store,Restaurant,Sports Bar,Bank,Bagel Shop,Supermarket


## Cluster Areas

In [24]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(["PostalCode", "Borough", "Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 1, 1, 1, 2, 1, 1, 1, 2, 1])

In [25]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
toronto_merged = merged_data.copy()

# add clustering labels
toronto_merged["Cluster Labels"] = kmeans.labels_

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(["Borough", "Neighborhoods"], 1).set_index("PostalCode"), on="PostalCode")

print(toronto_merged.shape)
toronto_merged.head() # check the last columns!

(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Restaurant,Theater,Café,Dessert Shop,Distribution Center
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Diner,Yoga Studio,College Cafeteria,Beer Bar,Smoothie Shop,Sandwich Place,Burrito Place,Café,College Auditorium
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Clothing Store,Coffee Shop,Japanese Restaurant,Cosmetics Shop,Italian Restaurant,Café,Hotel,Bubble Tea Shop,Fast Food Restaurant,Electronics Store
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Café,Coffee Shop,Clothing Store,Cocktail Bar,Restaurant,Cosmetics Shop,American Restaurant,Breakfast Spot,Italian Restaurant,Creperie
4,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Asian Restaurant,Trail,Pub,Neighborhood,Health Food Store,Dog Run,Distribution Center,Doner Restaurant,Cupcake Shop,Discount Store


In [26]:
# sort the results by Cluster Labels
print(toronto_merged.shape)
toronto_merged.sort_values(["Cluster Labels"], inplace=True)
toronto_merged

(39, 16)


,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Restaurant,Theater,Café,Dessert Shop,Distribution Center
20,M4P,Central Toronto,Davisville North,43.712751,-79.390197,1,Convenience Store,Breakfast Spot,Hotel,Food & Drink Shop,Sandwich Place,Department Store,Gym / Fitness Center,Park,Doner Restaurant,Dog Run
21,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307,1,Park,Jewelry Store,Trail,Sushi Restaurant,Yoga Studio,Dance Studio,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
24,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678,1,Café,Sandwich Place,Coffee Shop,Park,BBQ Joint,Pharmacy,Pizza Place,Pub,Middle Eastern Restaurant,Metro Station
25,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325,1,Gift Shop,Breakfast Spot,Dog Run,Dessert Shop,Restaurant,Bookstore,Movie Theater,Bar,Cuban Restaurant,Italian Restaurant
26,M4S,Central Toronto,Davisville,43.704324,-79.388790,1,Dessert Shop,Sandwich Place,Pizza Place,Coffee Shop,Italian Restaurant,Café,Gym,Sushi Restaurant,Pharmacy,Seafood Restaurant
27,M5S,Downtown Toronto,"University of Toronto, Harbord",43.662696,-79.400049,1,Café,Restaurant,Sandwich Place,Bar,Japanese Restaurant,Bookstore,Bakery,Italian Restaurant,Beer Bar,Beer Store
37,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,1,Coffee Shop,Japanese Restaurant,Restaurant,Sushi Restaurant,Gay Bar,Yoga Studio,Pub,Bubble Tea Shop,Men's Store,Mediterranean Restaurant
28,M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450,1,Pizza Place,Café,Coffee Shop,Sushi Restaurant,Pub,Italian Restaurant,Yoga Studio,Boutique,Spa,Indie Movie Theater
31,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049,1,Coffee Shop,Pub,Sushi Restaurant,Pizza Place,Liquor Store,Restaurant,Sports Bar,Bank,Bagel Shop,Supermarket


#### Finally, let's visualize the resulting clusters

In [27]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Borough'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

#### Cluster-1:

In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,0,Coffee Shop,Bakery,Pub,Park,Breakfast Spot,Restaurant,Theater,Café,Dessert Shop,Distribution Center


#### Cluster-2:

In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Central Toronto,1,Convenience Store,Breakfast Spot,Hotel,Food & Drink Shop,Sandwich Place,Department Store,Gym / Fitness Center,Park,Doner Restaurant,Dog Run
21,Central Toronto,1,Park,Jewelry Store,Trail,Sushi Restaurant,Yoga Studio,Dance Studio,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
24,Central Toronto,1,Café,Sandwich Place,Coffee Shop,Park,BBQ Joint,Pharmacy,Pizza Place,Pub,Middle Eastern Restaurant,Metro Station
25,West Toronto,1,Gift Shop,Breakfast Spot,Dog Run,Dessert Shop,Restaurant,Bookstore,Movie Theater,Bar,Cuban Restaurant,Italian Restaurant
26,Central Toronto,1,Dessert Shop,Sandwich Place,Pizza Place,Coffee Shop,Italian Restaurant,Café,Gym,Sushi Restaurant,Pharmacy,Seafood Restaurant
27,Downtown Toronto,1,Café,Restaurant,Sandwich Place,Bar,Japanese Restaurant,Bookstore,Bakery,Italian Restaurant,Beer Bar,Beer Store
37,Downtown Toronto,1,Coffee Shop,Japanese Restaurant,Restaurant,Sushi Restaurant,Gay Bar,Yoga Studio,Pub,Bubble Tea Shop,Men's Store,Mediterranean Restaurant
28,West Toronto,1,Pizza Place,Café,Coffee Shop,Sushi Restaurant,Pub,Italian Restaurant,Yoga Studio,Boutique,Spa,Indie Movie Theater
31,Central Toronto,1,Coffee Shop,Pub,Sushi Restaurant,Pizza Place,Liquor Store,Restaurant,Sports Bar,Bank,Bagel Shop,Supermarket
32,Downtown Toronto,1,Airport Service,Airport Lounge,Airport Terminal,Coffee Shop,Harbor / Marina,Boutique,Boat or Ferry,Rental Car Location,Plane,Sculpture Garden


#### Cluster-3:

In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,2,Coffee Shop,Clothing Store,Yoga Studio,Sporting Goods Shop,Fast Food Restaurant,Diner,Metro Station,Mexican Restaurant,Miscellaneous Shop,Chinese Restaurant
4,East Toronto,2,Asian Restaurant,Trail,Pub,Neighborhood,Health Food Store,Dog Run,Distribution Center,Doner Restaurant,Cupcake Shop,Discount Store
10,Downtown Toronto,2,Coffee Shop,Aquarium,Hotel,Café,Brewery,Scenic Lookout,Fried Chicken Joint,Restaurant,Sporting Goods Shop,Bakery
8,Downtown Toronto,2,Coffee Shop,Café,Clothing Store,Restaurant,Hotel,Deli / Bodega,Gym,Thai Restaurant,Pizza Place,Bar


#### Cluster-4:

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Downtown Toronto,3,Café,Coffee Shop,Vegetarian / Vegan Restaurant,Mexican Restaurant,Vietnamese Restaurant,Bar,Grocery Store,Pizza Place,Dumpling Restaurant,Burger Joint


#### Cluster-5:

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,West Toronto,4,Mexican Restaurant,Café,Thai Restaurant,Grocery Store,Flea Market,Bakery,Italian Restaurant,Speakeasy,Cajun / Creole Restaurant,Fried Chicken Joint


### Observations:

Most of the neighborhoods fall into Cluster 2. Then Cluster 3 has comparatively less neighborhoods. The remaining clusters has very few neighborhoods